# Not Diamond

> [Not Diamond](https://www.notdiamond.ai/) automatically determines which model is best-suited to respond to any query, for improved quality, and reduced cost and latency. 

Langchain supports automatic model routing to all [models supported by Not Diamond](https://notdiamond.readme.io/docs/llm-models) using `ChatNotDiamond`. This notebook covers how to get started with automatic model routing using Langchain + the Not Diamond I/O library. 

## Setup

Install the `notdiamond[create]` package that includes `langchain_community` and other langchain modules for generating LLM responses from various providers.

In [ ]:
%pip install --upgrade --quiet notdiamond[create]

Create your [Not Diamond API key](https://app.notdiamond.ai/keys). Set `NOTDIAMOND_API_KEY` environment variable or use the `notdiamond_api_key` keyword argument. Additionally, provide API keys for all providers that you want to route between.

In [1]:
import os

os.environ["NOTDIAMOND_API_KEY"] = "YOUR_NOTDIAMOND_API_KEY"
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"

Initialize a chat model with routing capabilities using `ChatNotDiamond`. You should include your desired providers and models to route between using `llm_configs`.

In [6]:
from langchain_community.chat_models import ChatNotDiamond

models = ['openai/gpt-4o', 'anthropic/claude-3-5-sonnet-20240620']
chat = ChatNotDiamond(llm_configs=models)

## Usage

`ChatNotDiamond` supports all methods of `ChatModel` including async APIs.

You can also use functionality of `invoke`, `generate`, and `stream`.

In [7]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]

response = chat(messages)
print("Not Diamond session ID: ", response.response_metadata['session_id'])  # Unique ID of Not Diamond's recommendation
print("LLM called: ", response.response_metadata['model_name'])  # LLM routed to
print("Response:", response.content) # LLM response

Not Diamond session ID:  a0482264-3d71-48fb-b043-c9d428718905
LLM called:  anthropic/claude-3-5-sonnet-20240620
Response: The translation of "I love programming" from English to French is:

J'adore la programmation.

You can also say:

J'aime programmer.

Both translations are correct and convey the same meaning. The first one literally means "I love programming," while the second one translates more closely to "I love to program."


## `ChatNotDiamond` also supports async and streaming functionality

In [8]:
await chat.agenerate([messages])

LLMResult(generations=[[ChatGeneration(text='Here\'s the translation of "I love programming" from English to French:\n\nJ\'adore programmer.\n\nYou could also say:\n\nJ\'aime la programmation.\n\nBoth versions are correct and convey the same meaning. The first one is more direct and literally translates to "I love to program," while the second one translates to "I love programming" as a noun.', generation_info={'finish_reason': None}, message=AIMessage(content='Here\'s the translation of "I love programming" from English to French:\n\nJ\'adore programmer.\n\nYou could also say:\n\nJ\'aime la programmation.\n\nBoth versions are correct and convey the same meaning. The first one is more direct and literally translates to "I love to program," while the second one translates to "I love programming" as a noun.', response_metadata={'session_id': '0c5b5360-8821-4650-b80b-bee12e3c4d1b', 'model_name': 'anthropic/claude-3-5-sonnet-20240620', 'input_tokens': 20, 'output_tokens': 85, 'total_tokens

In [9]:
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

chat = ChatNotDiamond(
    llm_configs=models,
    streaming=True,
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)
chat(messages)

Here's the translation of "I love programming" in French:

J'adore la programmation.

A few notes:
- "J'adore" means "I love" or "I adore"
- "La programmation" is "programming" in French, with "la" being the feminine definite article

You could also say:
J'aime programmer.

This version uses "J'aime" (I like/love) and the verb form "programmer" (to program) instead of the noun form.

AIMessage(content='Here\'s the translation of "I love programming" in French:\n\nJ\'adore la programmation.\n\nA few notes:\n- "J\'adore" means "I love" or "I adore"\n- "La programmation" is "programming" in French, with "la" being the feminine definite article\n\nYou could also say:\nJ\'aime programmer.\n\nThis version uses "J\'aime" (I like/love) and the verb form "programmer" (to program) instead of the noun form.', id='run-30f9d9bf-e52a-49a6-880f-e6c59643455a')

## Tool Calling

`ChatNotDiamond` also provides structured output and function calling use cases for [models that support these features](https://notdiamond.readme.io/docs/function-calling).

In [10]:
from pprint import pprint

from langchain_community.chat_models import ChatNotDiamond
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool

# Defining our tools
@tool
def add(a: int, b: int) -> int:
    "Adds a and b."
    return a + b
  
@tool
def multiply(a: int, b: int) -> int:
    "Multiplies a and b."
    return a * b

# Define the models to route between and the client
models = ['openai/gpt-4o', 'anthropic/claude-3-5-sonnet-20240620', 'google/gemini-1.5-pro-latest']
chat = ChatNotDiamond(llm_configs=models)

# Binding the add and multiply tools to the client
chat.bind_tools([add, multiply]) 

messages = [
    SystemMessage(content="You are a bot that helps with math calculations using provided functions. For every question that is asked, call the correct function."),
    HumanMessage(content="What is 10 * 12?")
]

# Invoking the chat
response = chat(messages)

print("Not Diamond session ID: ", response.response_metadata['session_id'])  # Unique ID of Not Diamond's recommendation
print("LLM called: ", response.response_metadata['model_name'])  # LLM routed to
pprint(f"Response: {response.content}") # LLM response

Not Diamond session ID:  ec5db2c4-2088-4d74-a2db-49ec9d6e82d1
LLM called:  anthropic/claude-3-5-sonnet-20240620
('Response: [{\'text\': \'To calculate 10 * 12, I can use the "multiply" '
 "function that\\'s available to me. Let me call that function for you.', "
 "'type': 'text'}, {'id': 'toolu_013BFmigdUjH1SaaJiCW83J4', 'input': {'a': 10, "
 "'b': 12}, 'name': 'multiply', 'type': 'tool_use'}]")
